# Ejercicio Práctica 7: curvas de aprendizaje en Machine Learning: the California housing problem 

Una curva de aprendizaje en Machine Learning simplemente es una gráfica donde se representa las epochs (eje horizontal) frente a la función objetivo (eje vertical).

En este ejercicio vermos cómo usar las curvas de aprendizaje para analizar el comportamiento de varios algoritmos de optimización. Para ello, estudiaremos el llamado **California housing problem**, el cual es un problema de regresión que consiste en predecir el valor que debe tener una casa en California dependiendo de varias características. La descripción de los datos es la siguiente:

**Labels**

'MedHouseVal' = Mediana de viviendas para familias en una manzana (medido en dólares estadounidenses)

**Features**

'MedInc' = Media de ingresos para grupos familiares en una manzana (medido en decenas de miles de dólares estadounidenses)

'HouseAge' = Antigüedad media de una casa 

'AveRooms' = Media de habitaciones en un bloque de casas

'AveBedrms' = Media de habitaciones en un barrio

'Population' = Cantidad total de personas que residen en un barrio

'AveOccup' = media de casa ocupadas

'Latitude' = latitud geográfica de la casa

'Longitude' = longitud geográfica de la casa

Estas dos últimas características hacen referencia a la localización de la casa

Los datos están almacenados es una basa de datos de **scikitlearn**. Con las siguientes líneas de código los cargamos.

In [1]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

housing


{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

![En este gráfico puedes ver la distribución de casas y precios](..\data\california_housing.png)

Las siguientes líneas de código hacen la separación entre datos de entrenamiento, validación y test.

También normalizan los datos. Esta es una técnica usual en Machine Learning que trata de evitar sesgos en los datos de partida. Para cada una de las 8 columnas de características, la normalización de los datos se hace de manera stándard del siguiente modo:

1) Se calculan la media $\mu_j$ y la desviación típica $\sigma_j$, $1\leq j\leq 8$.

2) Si $x^j$ denota la columna $j-$ésima, entonces dicha columna se transforma en 

$$
\hat{x}^j = \frac{x^j - \mu_j}{\sigma_j}
$$

con lo cual los nuevos datos tienen media cero y varianza 1.

Las siguientes líneas de código hacen el trabajo.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)



Nótese que ha aparecido un nuevo conjunto de datos **X_valid**. Se trata de un subconjunto de datos que se usa para tunear hiperparámetros del modelo, por ejemplo el número de capas ocultas, neuronas por capa, etc. Por tanto, **X_valid** se usa durante el entrenamiento. Es un subconjunto de los datos de entrenamiento.

Imprimer por pantalla las dimensiones de los conjuntos **X_train**, **X_val** y **X_test**.

In [3]:
# Completar aquí

# --------------------


A continuación cargamos **tensorflow** y **numpy**. Asímismo, fijamos semillas en **numpy** y **tensorflow** para que la reproducción de procesos aleatorios de el mismo resultado.

In [4]:
import numpy as np
import tensorflow as tf

In [5]:

np.random.seed(42)
tf.random.set_seed(42)

A continuación, construimos el modelo de predicción (la red neuronal). Se trata de un MultiLayerPerceptron (MLP) con $8$ canales de entrada, una sóla capa oculta de $30$ neuronas, y una salida escalar. Es decir
$$
NN(x,\theta) =  \sum_{j=1}^{30} a_j\sigma \left(\omega x + b\right)_j + b_{output}, \quad \theta = (\omega; b) 
$$
donde $\sigma$ es la función de activación, la cual se aplica componente a componente, $\omega$ es una matriz de tamaño $30\times 8$, $b$ es un vector bias de tamaño $30$, y $b_{output}$ es un bias de salida escalar. Por tanto, nuestro modelo tiene $30\times 8 + 30 + 30 +1 = 301 $ parámetros de entrenamiento.

In [6]:
from tensorflow import keras


In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer='glorot_uniform',
                        input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.summary()

Explica con tus propias palabras lo que hacen las líneas de código de la celda anterior. Para ello consulta la  [API de Keras](https://keras.io/api/)

In [8]:
# Completar aquí

# --------------------


Configuramos el modelo para el entrenamiento. Elegimos como función objetivo (loss function) el error cuadrático medio, y como optimizador el gradiente estocástico sencillo learning rate = 0.01

In [9]:

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

Entrenamos el modelo con el [método fit](https://keras.io/api/models/model_training_apis/)

In [10]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

¿Se ha usado mini-batch en el método de gradiente estocástico anterior? En caso afirmativo, ¿cúantos datos contiene cada mini-batch?

Te puede ayudar a responder estas preguntas los resultados que obtienes de ejecutar la celda siguiente, que también has de explicar.

In [11]:
print(history.params)
print(history.history)

In [12]:
# Completar aquí

# --------------------


A continuación nos ocupamos de analizar la evolución del algoritmo de optimización (training process).
Para ello usaremos los módulos **pandas** y **matplotlib** 

In [13]:

import pandas as pd
import matplotlib.pyplot as plt

In [19]:
loss_history = pd.DataFrame(history.history)
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(loss_history['loss'], label='Loss')
ax.plot(loss_history['val_loss'], label='Validation loss')
ax.grid()
ax.set_ylim(0, 1)
ax.set_xlabel('epochs')
ax.set_ylabel('loss function')
ax.legend();

Finalmente, evaluamos el error cuadrático medio sobre los datos test con  [model.evaluate](https://keras.io/api/models/model_training_apis/) y hacemos predicciones sobre conjuntos de datos con [model.predict](https://keras.io/api/models/model_training_apis/)

In [16]:
mse_test = model.evaluate(X_test, y_test)
print(f"MSE_test = {mse_test}")
X_new = X_test[:3]
print(f"X_new = { X_new}")
y_pred = model.predict(X_new)
print(f"y_predict = {y_pred}")

Para acabar la práctica, vamos a entrenar el modelo con otro algoritmo de optimización. Para ello:

1) Crea de nuevo el modelo y llámale **model_2**.

2) Configura el modelo de entrenamiento  **model_2** para lo cual debes elegir algún otro algoritmo de optimización (el que quieras) de los disponibles en el método **model.compile** 

3) Entrena **model_2** con mini-batches que contengan $64$ datos y para $50$ epochs. 

Los resultados que se muestran a continuación se corresponden con el algoritmo **Adam**.

In [17]:
# Completar aquí

# --------------------


Finalmente, dibuja las curvas de aprendizaje para tu modelo.

In [21]:
# Completar aquí

# --------------------
